# Database of studies
In this notebook, a database of students, teachers and courses is constructed with SQLite.


In [80]:
import os
import sqlite3
import pandas as pd

db = sqlite3.connect("courses.db")
db.isolation_level = None

Create tables used in the database

In [81]:
def create_tables():
    db.execute("CREATE TABLE Teachers (id INTEGER PRIMARY KEY, name TEXT)")
    db.execute("CREATE TABLE Courses (id INTEGER PRIMARY KEY, name TEXT, credits INTEGER)")
    db.execute("CREATE TABLE Students (id INTEGER PRIMARY KEY, name TEXT)")
    db.execute("CREATE TABLE CourseTeachers (course_id INTEGER REFERENCES Courses, teacher_id INTEGER REFERENCES Teachers)")
    db.execute("CREATE TABLE CompletedCourses (student_id INTEGER REFERENCES Students, course_id INTEGER REFERENCES Courses, date DATE, grade INTEGER)")
    db.execute("CREATE TABLE Groups (id INTEGER PRIMARY KEY, name TEXT)")
    db.execute("CREATE TABLE GroupStudents (group_id INTEGER REFERENCES Groups, student_id INTEGER REFERENCES Students)")
    db.execute("CREATE TABLE GroupTeachers (group_id INTEGER REFERENCES Groups, teacher_id INTEGER REFERENCES Teachers)")

Create functions for adding information to the tables and retrieve information.

In [82]:
# Add teacher to database
def create_teacher(name):
    opettaja = db.execute("INSERT INTO Teachers (name) VALUES (?)", [name])
    return opettaja.lastrowid

# Add course
def create_course(name, credits, teacher_ids):
    course = db.execute("INSERT INTO Courses (name, credits) VALUES (?, ?)", [name, credits])
    for t_id in teacher_ids:
        db.execute("INSERT INTO CourseTeachers (course_id, teacher_id) VALUES (?, ?)", [course.lastrowid ,t_id])
    return course.lastrowid

# Add student
def create_student(name):
    student = db.execute("INSERT INTO Students (name) VALUES (?)", [name])
    return student.lastrowid

# Add course credits and grade for a student
def add_credits(student_id, course_id, date, grade):
    db.execute("INSERT INTO CompletedCourses (student_id, course_id, date, grade) VALUES (?, ?, ?, ?)", [student_id, course_id, date, grade])

# Add a group to the database
def create_group(name, teacher_ids, student_ids):
    group = db.execute("INSERT INTO Groups (name) VALUES (?)", [name])
    for t_id in teacher_ids:
        db.execute("INSERT INTO GroupTeachers (group_id, teacher_id) VALUES (?, ?)", [group.lastrowid, t_id])
    for s_id in student_ids:
        db.execute("INSERT INTO GroupStudents (group_id, student_id) VALUES (?, ?)", [group.lastrowid, s_id])

# Get courses that a teacher lectures (alphabetically ordered)
def courses_by_teacher(teacher_name):
    courses = db.execute("SELECT C.name FROM Teachers T, Courses C, CourseTeachers CT WHERE T.id = CT.teacher_id AND C.id = CT.course_id AND T.name = ? ORDER BY C.name",[teacher_name]).fetchall()
    courses = [i[0] for i in courses]
    return courses

# Get credits given by teacher
def credits_by_teacher(teacher_name):
    credits = db.execute("SELECT SUM(C.credits) FROM Teachers T, Courses C, CourseTeachers CT, CompletedCourses CC WHERE T.id = CT.teacher_id AND C.id = CT.course_id AND CC.course_id = C.id AND T.name = ?", [teacher_name]).fetchone()
    return credits[0]

# Get completed courses by a student
def courses_by_student(student_name):
    courses = db.execute("SELECT C.name, CC.grade FROM Courses C, Students S, CompletedCourses CC WHERE S.id = CC.student_id AND C.id = CC.course_id AND S.name = ? ORDER BY C.name", [student_name]).fetchall()
    return courses

# Get credits done in a year
def credits_by_year(year):
    year = str(year)
    credits = db.execute("SELECT SUM(C.credits) FROM CompletedCourses CC, Courses C WHERE C.id = CC.course_id AND SUBSTR(CC.date,1,4) = ?", [year]).fetchone()
    return credits[0]

# Get distribution of grade for a course
def grade_distribution(course_name):
    grades = db.execute("SELECT CC.grade FROM Courses C LEFT JOIN CompletedCourses CC ON C.id = CC.course_id WHERE C.name = ?", [course_name]).fetchall()
    grades = [i[0] for i in grades]
    grades = dict({1: grades.count(1), 2: grades.count(2), 3: grades.count(3), 4: grades.count(4), 5: grades.count(5)})
    return grades

# Get a list of information of the courses
def course_list():
    courses = db.execute("SELECT C.name, COUNT(DISTINCT CT.teacher_id), COUNT(DISTINCT CC.student_id) FROM Courses C LEFT JOIN CourseTeachers CT ON C.id = CT.course_id LEFT JOIN CompletedCourses CC ON C.id = CC.course_id GROUP BY C.name ORDER BY C.name").fetchall()
    return courses

# Get a list of teachers and their courses
def teacher_list():
    teachers = db.execute("SELECT T.name FROM Teachers T").fetchall()
    teachers.sort()
    courses = []
    for t in teachers:
        t_courses = db.execute("SELECT C.name FROM Teachers T LEFT JOIN CourseTeachers CT ON T.id = CT.teacher_id LEFT JOIN Courses C ON CT.course_id = C.id WHERE T.name = ? ORDER BY T.name, C.name", [t[0]]).fetchall()
        t_courses = [i[0] for i in t_courses]
        courses.append((t[0],t_courses))

    return courses

# Get group members
def group_people(group_name):
    teachers = db.execute("SELECT T.name FROM Groups G, GroupTeachers GT, Teachers T WHERE G.id = GT.group_id AND GT.teacher_id = T.id AND G.name = ?", [group_name]).fetchall()
    students = db.execute("SELECT S.name FROM Groups G, GroupStudents GS, Students S WHERE G.id = GS.group_id AND GS.student_id = S.id AND G.name = ?", [group_name]).fetchall()
    teachers = [i[0] for i in teachers]
    students = [i[0] for i in students]
    members = teachers+students
    members.sort()
    return members

# Get the amount of credits in groups
def credits_in_groups():
    credits = db.execute("SELECT G.name, IFNULL(SUM(C.credits),0) FROM Groups G LEFT JOIN GroupStudents GS ON G.id = GS.group_id LEFT JOIN CompletedCourses CC ON GS.student_id = CC.student_id LEFT JOIN Courses C ON CC.course_id = C.id GROUP BY G.name ORDER BY G.name").fetchall()
    return credits

# Get common groups of a teacher and a student
def common_groups(teacher_name, student_name):
    group = db.execute("SELECT G.name FROM Groups G, GroupStudents GS, GroupTeachers GT, Students S, Teachers T WHERE G.id = GS.group_id AND G.id = GT.group_id AND GS.student_id = S.id AND GT.teacher_id = T.id AND S.name = ? AND T.name = ?", [student_name, teacher_name]).fetchall()
    group = [i[0] for i in group]
    return group


Add information to database.

In [83]:
# Create tables and add information
create_tables()

# Add teachers
t1 = create_teacher("Pentti Penalainen")
t2 = create_teacher("Matti Matikainen")
t3 = create_teacher("Pirjo Pirkkala")
t4 = create_teacher("Päivi Päiväinen")
t5 = create_teacher("Lauri Laurilainen")

# Add courses
c1 = create_course("Python Basics", 5, [t1, t3])
c2 = create_course("Data Analysis", 8, [t1, t2, t5])
c3 = create_course("Machine Learning", 5, [t2, t5])
c4 = create_course("Data Processing and Visualization", 10, [t3, t4])
c5 = create_course("Statistical Analysis", 6, [t5]) # Ei opiskelijoita

# Add students
s1 = create_student("Henri Heikkinen")
s2 = create_student("Late Mallikas")
s3 = create_student("Laura Noheva")
s4 = create_student("Jenna Jauhoinen")
s5 = create_student("Arttu Auttavainen") # Ei kursseja

# Add credits to students
add_credits(s1, c1, "2020-01-10", 1)
add_credits(s1, c2, "2021-05-01", 2)
add_credits(s1, c4, "2021-04-02", 3)
add_credits(s2, c1, "2021-03-01", 5)
add_credits(s2, c2, "2022-09-08", 5)
add_credits(s3, c3, "2022-09-10", 3)
add_credits(s4, c3, "2022-11-01", 3)
add_credits(s4, c4, "2020-11-29", 5)

# Add groups
create_group("Urheilijat", [t1, t2], [s1, s2, s3, s5])
create_group("Koodarit", [t4], [s2, s4, s5])
create_group("Matemaatikot", [], [s5])
create_group("Fyysikot", [t1, t2, t3], [s2, s3, s4, s5])

Let's visualize some examples of the created tables

In [84]:
print("Teachers")
pd.read_sql_query("SELECT * FROM Teachers", db).head()

Teachers


,id,name
0,1,Pentti Penalainen
1,2,Matti Matikainen
2,3,Pirjo Pirkkala
3,4,Päivi Päiväinen
4,5,Lauri Laurilainen


In [85]:
print("Students")
pd.read_sql_query("SELECT * FROM Students", db).head()

Students


,id,name
0,1,Henri Heikkinen
1,2,Late Mallikas
2,3,Laura Noheva
3,4,Jenna Jauhoinen
4,5,Arttu Auttavainen


In [86]:
print("Courses")
pd.read_sql_query("SELECT * FROM Courses", db).head()

Courses


,id,name,credits
0,1,Python Basics,5
1,2,Data Analysis,8
2,3,Machine Learning,5
3,4,Data Processing and Visualization,10
4,5,Statistical Analysis,6


Let's get some information from the database

In [87]:
print(f"Courses given by Pentti Penalainen: {courses_by_teacher('Pentti Penalainen')}")
print(f"Amount of credits given by Pentti Penalainen: {credits_by_teacher('Pentti Penalainen')}")
print(f"Completed courses by Henri Heikkinen: {courses_by_student('Henri Heikkinen')}")

print("")
print(f"Credits in 2020: {credits_by_year(2020)}")
print(f"Credits in 2021: {credits_by_year(2021)}")
print(f"Credits in 2022: {credits_by_year(2022)}")
print(f"Grade distribution of Python Basics: {grade_distribution('Python Basics')}")
print(f"Grade distribution of Machine Learning: {grade_distribution('Machine Learning')}")

print("")
print("List of courses, number of teachers and number of completions:")
print(course_list())
print("")
print("List teachers and their courses:")
print(teacher_list())

print("")
print(f"Members in group Koodarit: {group_people('Koodarit')}")
print("Credits in each group:")
print(credits_in_groups())
print(f"Common groups of Pentti Penalainen and Late Mallikas: {common_groups('Pentti Penalainen','Laura Noheva')}")


Courses given by Pentti Penalainen: ['Data Analysis', 'Python Basics']
Amount of credits given by Pentti Penalainen: 26
Completed courses by Henri Heikkinen: [('Data Analysis', 2), ('Data Processing and Visualization', 3), ('Python Basics', 1)]

Credits in 2020: 15
Credits in 2021: 23
Credits in 2022: 18
Grade distribution of Python Basics: {1: 1, 2: 0, 3: 0, 4: 0, 5: 1}
Grade distribution of Machine Learning: {1: 0, 2: 0, 3: 2, 4: 0, 5: 0}

List of courses, number of teachers and number of completions:
[('Data Analysis', 3, 2), ('Data Processing and Visualization', 2, 2), ('Machine Learning', 2, 2), ('Python Basics', 2, 2), ('Statistical Analysis', 1, 0)]

List teachers and their courses:
[('Lauri Laurilainen', ['Data Analysis', 'Machine Learning', 'Statistical Analysis']), ('Matti Matikainen', ['Data Analysis', 'Machine Learning']), ('Pentti Penalainen', ['Data Analysis', 'Python Basics']), ('Pirjo Pirkkala', ['Data Processing and Visualization', 'Python Basics']), ('Päivi Päiväinen'

In [88]:
db.close()